In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "sh2orc/Llama-3.1-Korean-8B-Instruct"

SYSTEM_PROMPT = """
당신은 지휘관의 결심을 돕는 군사 의사결정 보조 AI입니다.

역할:
- 주어진 전장 상황, 아군 전력 상태, 임무를 토대로 가능한 여러 개의 행동방안(COA)을 제안합니다.
- 각 COA에 대해: 의도, 개념(주요 행동), 필요한 전력, 장점, 단점, 주요 위험요인, 전제조건을 정리합니다.
- 구체적인 표적/좌표/사격지시 등 "직접적인 교전 지시"는 절대 하지 않습니다.
- 항상 "최종 결심은 인간 지휘관이 한다"는 점을 명확히 합니다.

출력 형식:
1. 상황 요약
2. COA 1
3. COA 2
4. COA 3
5. COA 비교 및 고려사항

가능한 한 간결하고 실무적인 한국어로 답변하세요.
""".strip()


def load_model():
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16,
    )

    return tokenizer, model


def generate_response(tokenizer, model, user_input: str, max_new_tokens: int = 512):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_input},
    ]

    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Remove prompt prefix if it appears in the decoded text
    if decoded.startswith(prompt):
        decoded = decoded[len(prompt):]

    return decoded.strip()


if __name__ == "__main__":
    tokenizer, model = load_model()

    user_input = """
적 전차 2대와 전투기 1대가 관측되었고, 전체 추정 전력은 약 2개 여단 규모로 평가된다.
우리 부대는 기계화 보병대대 1개(전투력 80%), 보병중대 1개(전투력 60%), 대전차 화력은 중간 수준,
방공전력은 제한적이다. 상급부대 임무는 '12시간 동안 적의 돌파를 지연하고 전투력을 보존할 것'이다.
지형은 개활지와 소규모 구릉이 혼재하며, 우리 후방에는 중요한 교량 1개가 있다.

이 상황에서 고려할 수 있는 행동방안(COA)을 제시해 줘.
""".strip()

    print("=== 지휘관 입력 ===")
    print(user_input)
    print("\n=== AI 응답 ===\n")
    print(generate_response(tokenizer, model, user_input))
